In [1]:
from langchain.llms import GooglePalm
import config

In [2]:
llm = GooglePalm(google_api_key=config.GOOGLE_API_KEY, temperature=0.2)
poem = llm("Write a poem on Messi")
print(poem)

C:\Users\deves\anaconda3\envs\llm_env_google\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Lionel Messi**

Lionel Messi, the greatest of them all,
A magician with the ball,
A master of his craft,
A joy to watch for all.

He glides across the pitch,
Dribbling past defenders with ease,
Creating chances for his teammates,
And scoring goals from impossible angles.

He is a one-of-a-kind player,
A once-in-a-generation talent,
And he will be remembered as one of the greatest footballers of all time.

He is Lionel Messi,
The greatest of them all.


In [3]:
from langchain.utilities import SQLDatabase

db = SQLDatabase.from_uri(f"mysql+pymysql://{config.db_user}:{config.db_password}@{config.db_host}/{config.db_name}",sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [4]:
from langchain_experimental.sql import SQLDatabaseChain

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

q1 = db_chain("How many tshirts we have left for Van Huesen in extra large size and black color?")



> Entering new SQLDatabaseChain chain...
How many tshirts we have left for Van Huesen in extra large size and black color?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Van Huesen' AND color = 'Black' AND size = 'XL'
SQLResult: [(97,)]
Answer:97
> Finished chain.


In [5]:
q2 = db_chain("How much is the price of the inventory for all small size tshirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size tshirts?
SQLQuery:SELECT SUM(price) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('269'),)]
Answer:269
> Finished chain.


In [6]:
q2 = db_chain("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('12482'),)]
Answer:12482
> Finished chain.


In [7]:
q3 = db_chain("If we have to sell all the Levi's tshirts today with discount applied, How much revenue my store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi's tshirts today with discount applied, How much revenue my store will generate (post discounts)?
SQLQuery:SELECT SUM(price * (1 - pct_discount)) FROM discounts JOIN t_shirts ON discounts.t_shirt_id = t_shirts.t_shirt_id WHERE brand = 'Levi' AND CURDATE() BETWEEN start_date AND end_date

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'start_date' in 'where clause'")
[SQL: SELECT SUM(price * (1 - pct_discount)) FROM discounts JOIN t_shirts ON discounts.t_shirt_id = t_shirts.t_shirt_id WHERE brand = 'Levi' AND CURDATE() BETWEEN start_date AND end_date]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [8]:
q3 = db_chain("select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi' group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id")



> Entering new SQLDatabaseChain chain...
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi' group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) AS total_revenue FROM (SELECT sum(price*stock_quantity) AS total_amount, t_shirt_id FROM t_shirts WHERE brand = 'Levi' GROUP BY t_shirt_id) a LEFT JOIN discounts ON a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('26098.350000'),)]
Answer:26098.35
> Finished chain.


In [9]:
q4 = db_chain("How many white color Levi's tshirts do we have?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's tshirts do we have?
SQLQuery:SELECT count(*) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(2,)]
Answer:2
> Finished chain.


In [10]:
q4 = db_chain("SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('125'),)]
Answer:125
> Finished chain.


## Few Shots Learning

In [11]:
few_shots = [{'Question': "How many tshirts we have left for Van Huesen in extra large size and black color?", 'SQLQuery': "SELECT stock_quantity FROM t_shirts WHERE brand = 'Van Huesen' AND color = 'Black' AND size = 'XL'", 'SQLResult': "Result of SQL Query", 'Answer': q1['result']},
            {'Question': "How much is the price of the inventory for all small size tshirts?", 'SQLQuery': "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'", 'SQLResult': "Result of SQL Query", 'Answer': q2['result']},
            {'Question': "If we have to sell all the Levi's tshirts today with discount applied, How much revenue my store will generate (post discounts)?", 'SQLQuery': "select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi' group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id", 'SQLResult': "Result of SQL Query", 'Answer': q3['result']},
            {'Question': "How many white color Levi's tshirts do we have?", 'SQLQuery': "SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'", 'SQLResult': "Result of SQL Query", 'Answer': q4['result']}]

In [12]:
few_shots

[{'Question': 'How many tshirts we have left for Van Huesen in extra large size and black color?',
  'SQLQuery': "SELECT stock_quantity FROM t_shirts WHERE brand = 'Van Huesen' AND color = 'Black' AND size = 'XL'",
  'SQLResult': 'Result of SQL Query',
  'Answer': '97'},
 {'Question': 'How much is the price of the inventory for all small size tshirts?',
  'SQLQuery': "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
  'SQLResult': 'Result of SQL Query',
  'Answer': '12482'},
 {'Question': "If we have to sell all the Levi's tshirts today with discount applied, How much revenue my store will generate (post discounts)?",
  'SQLQuery': "select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi' group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id",
  'SQLResult': 'Result of SQL Query',
  'Answer': '26098.35'},


In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [14]:
to_vectorize = [' '.join(example.values()) for example in few_shots]
to_vectorize

["How many tshirts we have left for Van Huesen in extra large size and black color? SELECT stock_quantity FROM t_shirts WHERE brand = 'Van Huesen' AND color = 'Black' AND size = 'XL' Result of SQL Query 97",
 "How much is the price of the inventory for all small size tshirts? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' Result of SQL Query 12482",
 "If we have to sell all the Levi's tshirts today with discount applied, How much revenue my store will generate (post discounts)? select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi' group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id Result of SQL Query 26098.35",
 "How many white color Levi's tshirts do we have? SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White' Result of SQL Query 125"]

In [15]:
from langchain.vectorstores import Chroma

In [16]:
vector_stores = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [17]:
vector_stores

In [18]:
from langchain.prompts import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(vectorstore=vector_stores, k=2,)

example_selector.select_examples({'Question': "How many Adidas tshirts I have in my store?"})

[{'Answer': '97',
  'Question': 'How many tshirts we have left for Van Huesen in extra large size and black color?',
  'SQLQuery': "SELECT stock_quantity FROM t_shirts WHERE brand = 'Van Huesen' AND color = 'Black' AND size = 'XL'",
  'SQLResult': 'Result of SQL Query'},
 {'Answer': '125',
  'Question': "How many white color Levi's tshirts do we have?",
  'SQLQuery': "SELECT SUM(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
  'SQLResult': 'Result of SQL Query'}]

In [19]:
from langchain.chains.sql_database.prompt import _mysql_prompt, PROMPT_SUFFIX

print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [20]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [21]:
from langchain.prompts.prompt import PromptTemplate

prompt_template = PromptTemplate(input_variables=["Question", "SQLQuery", "SQLResult", "Answer"], template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}")

In [22]:
from langchain.prompts import FewShotPromptTemplate

few_shot_prompt = FewShotPromptTemplate(example_selector=example_selector, example_prompt=prompt_template, prefix=_mysql_prompt, suffix=PROMPT_SUFFIX, input_variables=["input", "table_info", "top_k"])

In [23]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [24]:
new_chain("If we have to sell all the Adidas and Puma tshirts today with discount applied, How much revenue my store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Adidas and Puma tshirts today with discount applied, How much revenue my store will generate (post discounts)?
SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from (select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand in ('Adidas','Puma') group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('22655.550000'),)]
Answer:22655.55
> Finished chain.


{'query': 'If we have to sell all the Adidas and Puma tshirts today with discount applied, How much revenue my store will generate (post discounts)?',
 'result': '22655.55'}